In [50]:
import os
from pathlib import Path
from mne.io import read_raw
from mne.channels import make_standard_montage
from mne_bids import BIDSPath, write_raw_bids
from tqdm.contrib import tzip
mne.set_log_level("Error")

In [ ]:
## subject 50033 is 50031 inside

root_dir = Path("/Volumes/Extreme_SSD/payam_data/Tide_data")
bids_dir = root_dir / "BIDS"
montage = make_standard_montage("easycap-M1")

sites = ["illinois"]

for site in sites:
    data_dir = root_dir / site
    if site in ["regensburg", "tuebingen"]:
        files = [
            f for f in data_dir.rglob("*.vhdr")
            if not any(p.startswith('.') for p in f.parts)
        ]

    if site == "illinois":
        files = [
            f for f in data_dir.rglob("*.cdt")
            if not any(p.startswith('.') for p in f.parts)
        ]
        for fname in files:
            dpo_file = fname.with_suffix('.cdt.dpo')
            dpa_file = fname.with_suffix('.cdt.dpa')
            if dpo_file.exists():
                dpo_file.rename(dpa_file)
    
    
    subject_ids = [f.stem[:5] for f in files]
    for fname, subject_id in tzip(files, subject_ids,
                                    total=len(subject_ids),
                                    desc=f"Converting {site}"):
        raw = read_raw(fname)
        bids_path = BIDSPath(
            subject=subject_id,
            session="01",
            task="rest",
            datatype="eeg",
            description=site,
            root=bids_dir
        )
        write_raw_bids(raw, bids_path=bids_path, overwrite=True)

Converting illinois:   0%|          | 0/24 [00:00<?, ?it/s]

In [90]:
## illinois
data_dir = root_dir / site
site = "illinois"
files = [
        f for f in data_dir.rglob("*.cdt")
        if not any(p.startswith('.') for p in f.parts)
    ]

for fname in files:
    dpo_file = fname.with_suffix('.cdt.dpo')
    dpa_file = fname.with_suffix('.cdt.dpa')
    if dpo_file.exists():
        dpo_file.rename(dpa_file)

subject_ids = [f.stem[:5] for f in files]

# for fname, subject_id in tzip(files, subject_ids,
#                                     total=len(subject_ids[:3]),
#                                     desc=f"Converting {site}"):
#         raw = read_raw(fname)
#         bids_path = BIDSPath(
#             subject=subject_id,
#             session="01",
#             task="rest",
#             datatype="eeg",
#             description=site,
#             root=bids_dir
#         )
#         write_raw_bids(raw, bids_path=bids_path, overwrite=True)

In [91]:
len(files)

74

In [ ]:
read_raw()


dpo_file = fname.with_suffix('.cdt.dpo')
                dpa_file = fname.with_suffix('.cdt.dpa')
                if dpo_file.exists():
                    dpo_file.rename(dpa_file)